In [3]:
# _dict__ provides a dictionary with attributes of the instance -> vars()

In [3]:
class Vector:
    def __init__(self,**coords):
        self.__dict__.update(coords)
        
    def __repr__(self):
        return 'Vector : ' + ', '.join( '{}={}'.format(key, value) for key, value in self.__dict__.items()  )
    
    # Overriding  
    def __setattr__(self, name, value):
        raise AttributeError("Can't set value {}".format(name))        
    
    def __delattr__(self, name, value):
        raise AttributeError("Can't delete value {}".format(name))
        
    def __getattr__(self, name):
        return self.__dict__[name]
        

In [4]:
v=Vector(a=2)


# adding poperty using setattr - will throw error 
try:
    setattr(v,'p',23)
except AttributeError as a:
    print(a)

# adding poperty using assignment - will also  throw error 
try:
    v.new_var = 12
except AttributeError as a:
    print(a)

    
# additng property using the __dict__ - still allowed (can store internally as _a and expect users will leave it as private attribute)
v.__dict__['z']=123
v.__dict__['a']=123

print('Vector :', v)

Can't set value p
Can't set value new_var
Vector : Vector : a=123, z=123


In [5]:
vars(v)

{'a': 123, 'z': 123}

In [52]:
class LoggingProxy:
    
    def __init__(self, target):
        # We're storing the target variable in the super class because we are going to override the getattribute 
        # (which will cause ALL the attribute lookups within the derived class to route to the overriden method)
        # meaning all self. calls will cause infinite recursion
        super().__setattr__('target', target)
        
        
    def __getattribute__(self, name):
        target = super().__getattribute__('target')
        try:
            # Delegating to getattr of the target
            value = getattr(target, name)
        except KeyError as e:
            raise AttributeError('{} could not forward request {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target                
            ))        
        print('Retrieved attribute {!r} = {!r} from {!r}'.format(name, value, target))
        return value
    
    def __setattr__(self, name, value):
        target = super().__getattribute__('target')
        try:
            # Delegating to setattr of the target
            setattr(target, name, value)
        except KeyError as e:
            raise AttributeError('{} could not forward request {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target                
            ))        
        print('Set attribute {!r} = {!r} on {!r}'.format(name, value, target))
        
    # overriding repr to route to underlying object
    def __repr__(self):
        target = super().__getattribute__('target')
        #repr_callable = getattr(target, '__repr__')
        return target.__repr__()

In [39]:
proxy = LoggingProxy(v)

Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12
Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12
Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12
Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12
Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12
Retrieved attribute '__class__' = <class '__main__.Vector'> from Vector : a=12, z=123, c=12, t=12


In [40]:
x = proxy.a
print(x)

Retrieved attribute 'a' = 12 from Vector : a=12, z=123, c=12, t=12
12


In [41]:
y = proxy.v3
print(y)

AttributeError: LoggingProxy could not forward request v3 to Vector : a=12, z=123, c=12, t=12

In [42]:
proxy.__dict__['t'] = 12

Retrieved attribute '__dict__' = {'a': 12, 'z': 123, 'c': 12, 't': 12} from Vector : a=12, z=123, c=12, t=12


In [43]:
proxy.__dict__

Retrieved attribute '__dict__' = {'a': 12, 'z': 123, 'c': 12, 't': 12} from Vector : a=12, z=123, c=12, t=12


{'a': 12, 'z': 123, 'c': 12, 't': 12}

In [44]:
# Above statement, we were able to fetch __dict__ from and set a value in it, overriding the setattr behaviour  

In [45]:
proxy.a = 0 # Raises attribute error - meaning a successful route to base class's setattr

AttributeError: Can't set value a

In [46]:
proxy.a

Retrieved attribute 'a' = 12 from Vector : a=12, z=123, c=12, t=12


12

In [47]:
# Attribute Lookup for speciual methods

proxy.__repr__() # this will invoke the underlying object's repr as the . (dot operator) is used

Retrieved attribute '__repr__' = <bound method Vector.__repr__ of Vector : a=12, z=123, c=12, t=12> from Vector : a=12, z=123, c=12, t=12


'Vector : a=12, z=123, c=12, t=12'

In [48]:
# But when we use the builtin repr function on proxy object, it does not invoke the underlying object's __repr__
repr(proxy)

'<__main__.LoggingProxy object at 0x0000000005572C50>'

In [53]:
# len(), iter(), repr() will all bypass getattribute due to performance reasons
# we will have to override these special funcitons to be able to route to the underlying objevt's respective method
proxy2 = LoggingProxy(v)
repr(proxy2)

'Vector : a=12, z=123, c=12, t=12'

In [65]:
# Where are the methods

v.__class__.__dict__['__repr__'](proxy2)
# The above line just called the repr function of v through it's class's dict to get the repr function 
# then I pass it the proxy object, 
# inside the repr of v, we are fetching the __dict__ object, that calls the getattribute function, which is 
# directing the calls to it's proxy object which also happens to be 'v', the dict is retrieved and then 
# pretty printed by the repr funciton of the Vector class
# he he he :P

Retrieved attribute '__dict__' = {'a': 12, 'z': 123, 'c': 12, 't': 12} from Vector : a=12, z=123, c=12, t=12


'Vector : a=12, z=123, c=12, t=12'

In [67]:
setattr(v.__class__, 'some_static_data', 123456)
print(Vector.some_static_data)

123456


In [74]:
# Trading Size for Dynamism with Slots
import sys
print('Vector: ',sys.getsizeof(v))
d = {}
print('Dict: ',sys.getsizeof(d))

Vector:  56
Dict:  240


In [76]:
class Resistor:
    def __init__(self, resistance_ohms, tolerance_percent, power_watts):
        self.resistance_ohms = resistance_ohms
        self.tolerance_percent = tolerance_percent
        self.power_watts = power_watts

class ResistorWithSlots:    
    # These are the attributes we expect - use with caution
    __slots__ = ['resistance_ohms', 'tolerance_percent', 'power_watts']
    def __init__(self, resistance_ohms, tolerance_percent, power_watts):
        self.resistance_ohms = resistance_ohms
        self.tolerance_percent = tolerance_percent
        self.power_watts = power_watts

In [84]:
r1 = Resistor(1.234, 123423.1, 1231)
r2 = ResistorWithSlots(1.234, 123423.1, 1231)

print('Object without slots :',sys.getsizeof(r1.__dict__))
print('Object with slots :',sys.getsizeof(r2))

Object without slots : 112
Object with slots : 64


In [85]:
# There is no __dict__ in the slots object
dir(r2)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'power_watts',
 'resistance_ohms',
 'tolerance_percent']